<a href="https://colab.research.google.com/github/yajima-yasutoshi/DataMining2024/blob/main/20241119/%E3%82%AF%E3%83%A9%E3%82%B9%E3%82%BF%E3%83%AA%E3%83%B3%E3%82%B0%E3%81%AE%E6%BC%94%E7%BF%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# クラスタリング演習


## データマイニング第8回（20241119）
https://github.com/yajima-yasutoshi/DataMining2024/tree/main/20241119


#本日の講義の目的
クラスタリング手法に関する演習を行う。


# 準備

In [ ]:
# インストール
!pip install japanize-matplotlib

## 必要なライブラリーのインポート

In [ ]:
# 必要なライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns

# Wineデータをつかった例

今回の演習では、sklearn に組み込まれているサンプルデータを用いる。
データの読み込みは、以下のように行う。

In [ ]:
from sklearn.datasets import load_wine
# データの読み込み
wine = load_wine()
df = pd.DataFrame(wine.data, columns=wine.feature_names)

データの確認を行う。主な確認ポイントは以下の通りである。


*   レコード数
*   項目数
*   各項目の型
*   欠損値の有無
*   外れ値（異常値）の有無


In [ ]:
df.info()

## 利用するデータ概要

Wineデータセットは、イタリアの同じ地域で栽培された3種類のワインに
含まれる成分を測定したデータ。
13の項目がある。

| 項目                          | 説明                                       |
|---------------------------------|--------------------------------------------|
| Alcohol（アルコール）           | ワインに含まれるアルコールの量。           |
| Malic Acid（リンゴ酸）          | ワインに含まれるリンゴ酸の量。             |
| Ash（灰分）                     | ワインの灰分の量。                         |
| Alcalinity of Ash（灰分のアルカリ度） | 灰分のアルカリ度。                   |
| Magnesium（マグネシウム）       | ワインに含まれるマグネシウムの量。         |
| Total Phenols（総フェノール）   | ワインに含まれるフェノール類の総量。       |
| Flavanoids（フラバノイド）      | ワインに含まれるフラバノイドの量。         |
| Nonflavanoid Phenols（非フラバノイドフェノール） | ワインに含まれる非フラバノイドフェノールの量。 |
| Proanthocyanins（プロアントシアニン） | ワインに含まれるプロアントシアニンの量。 |
| Color Intensity（色の強度）     | ワインの色の強度。                         |
| Hue（色相）                     | ワインの色相。                             |
| OD280/OD315 of Diluted Wines（希釈ワインのOD280/OD315） | 希釈されたワインのOD280/OD315の比率。 |
| Proline（プロリン）             | ワインに含まれるプロリンの量。             |

### 項目の選択

クラスタリングの処理を簡単にするため項目を限定する。

In [ ]:
df = pd.DataFrame(wine.data, columns=wine.feature_names)
# 相関行列を計算
correlation_matrix = df.corr().abs()
# sns.heatmap(correlation_matrix, cmap= sns.color_palette('coolwarm', 10), annot=True,fmt='.2f', vmin = -1, vmax = 1)

# 相関が0.6以上の変数を取得
high_corr_var = {}
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) >= 0.6:
            colname = correlation_matrix.columns[i]
            high_corr_var[colname] = abs(correlation_matrix.iloc[i, j])

# 相関性の高い変数を削除
df.drop(columns=high_corr_var.keys(), inplace=True)

# 結果の表示
# print("Remaining columns after removing highly correlated variables:")
print(df.columns)
df.info()

以降では、上記の9項目に限定して行う。

### 基本統計量を得る

数値型の項目の場合、最小値や最大値、平均や標準偏差といった数値を**基本統計量**
と呼び、分析の前には必ず確認する必要がある。

In [ ]:
df.describe()

### 数値型項目に関する確認事項

*   分布の形状（山が一つか？）、左右の偏りの確認。
*   分布の端が極端に離れていないか。外れ値がある可能性。
*   変数間の相関

### 外れ値（異常値）の確認

外れ値（異常値）とは、極端に大きな値や小さな値のことである。箱ひげ図を描くことで視覚的に確認ができる。

項目 alcohol に外れ値があるか、箱ひげ図を描いて確認する。下のxxxx部分は何か？

In [ ]:
sns.xxxxx(df['alcohol'])

項目 malic_acid に外れ値があるか、箱ひげ図を描いて確認する。

In [ ]:
sns.xxxx(df['malic_acid'])

箱ひげ図の上の横棒の値は、

> 箱の上＋1.5×箱の長さ

である。下の横棒は、

> 箱の下ー1.5×箱の長さ

また、箱の上は第三四分位、箱の下は第一四分位に相当する。

項目 malic_acid の上の横棒の値を計算する。

In [ ]:
q3 = df['malic_acid'].quantile(0.75)
q1 = df['malic_acid'].quantile(0.25)

# 以下続きを考え、上の横棒の値を計算する

malic_acid が上で求めた数値を超えるレコードを表示する。

In [ ]:
df[df['malic_acid'] > xxxxx]

あるいは、malic_acid の値で降順でソートしてもよい。

In [ ]:
df.sort_values( xxxxx )

なお、以降の分析は外れ値の除外は**行わず**に実行する。

標準化を実施する。

In [ ]:
# 標準化に必要なライブラリーのインポート
from sklearn.preprocessing import StandardScaler
# 前処理（データの標準化）
scaler = StandardScaler()

# 続きを考えよ。
X =

以下のコードでは、データが変数Xに格納されていることを前提としている（8行目）。もし変更が必要な場合は、適切に書き換えて実行する。

In [ ]:
# クラスタリングに必要なライブラリーのインポート
from sklearn.cluster import KMeans

# Elbow Methodによるハイパーパラメータチューニング
inertia = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=500, n_init='auto')
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)

In [ ]:
# エルボー図のプロット
elbow = pd.DataFrame( inertia, columns = ['Inertia'])
elbow['Num of clusters'] = range(1,11)
sns.lineplot(data = elbow, x='Num of clusters', y= 'Inertia')

エルボー図から最適なクラスター数を決定し、以下にセットする

In [ ]:
# エルボー図から最適なクラスター数を決定し、以下にセットする
best_n_clusters =

最適なクラスタ数で、クラスタリングを実行する。

In [ ]:
kmeans = KMeans(n_clusters=best_n_clusters, init='k-means++', max_iter=500, n_init='auto')
kmeans.fit(X)

# クラスタラベルをデータフレームに追加
df['cluster'] = kmeans.labels_

groupby を使ってクラスタで分類して、各項目の平均を計算する。

In [ ]:
# 各クラスタの特徴を調査（平均値）
# 以下のコードを完成させ実行する
df.groupby(XXXX

In [ ]:
# 結果の可視化（seabornを使用）
sns.scatterplot(x='alcohol', y='total_phenols', hue='cluster', data=df, palette='Set1')
plt.title('Clustering Results with Wine Data')
plt.show()